In [57]:
import logging
import tensorflow as tf
import keras

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

tf.__version__

'2.4.1'

In [58]:
import numpy as np
import pandas as pd
from pathlib import Path

import argparse
args = argparse.Namespace()
args.raw_train = pd.read_csv('../data/train.csv.zip')
args.raw_test = pd.read_csv('../data/test.csv.zip')
args.predictions_folder = Path('../predictions')
args.n_splits = 5
args.n_jobs = 3
args.seed = 101
args.val_fraction=0.1
args.epochs = 50

args.train = args.raw_train.iloc[:, 1:].copy()
args.labels = args.raw_train['label'].copy()
args.test = args.raw_test.copy()


args.predictions_folder.mkdir(parents=True, exist_ok=True) 

In [59]:
X = args.train.to_numpy().reshape(args.train.shape[0], 28, 28, 1)
y = pd.get_dummies(args.labels, prefix='label').to_numpy()
y_sparse = args.labels.to_numpy()
x = args.test.to_numpy().reshape(args.test.shape[0], 28, 28, 1)

In [60]:
t = tf.constant([[1, 2], [3, 4], [5, 6]])
tf.square(t)
keras.backend.transpose(t)
t + tf.cast(tf.constant(1.), tf.int32)
v = tf.Variable([[1, 2], [3, 4], [5, 6]])
v[0,0].assign(100)
tf.SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4])
tf.constant(['019A'])
# tf.strings.unicode_decode(tf.constant(['019A']), input_encoding='UTF-8')

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'019A'], dtype=object)>

### Custom training loops

In [8]:
len(X) // 32
32 * 1312

41984

In [65]:
# def shuffle(X, y):
#     x = np.array([0, 100, 200, 300, 400])
#     a = np.random.permutation(5)
#     print(a)
#     print(x[a])

# for i in range(5):
#     shuffle(None, None)
    
y_sparse[0:32]

array([1, 0, 1, 4, 0, 0, 7, 3, 5, 3, 8, 9, 1, 3, 3, 1, 2, 0, 7, 5, 8, 6,
       2, 0, 2, 3, 6, 9, 9, 7, 8, 9])

In [69]:
def build_simple_model():
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(28, 28)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'))
    model.add(keras.layers.Dense(10, activation='softmax')) # !
    return model
#     model.compile(loss='sparse_categorical_crossentropy', # !
#              optimizer=keras.optimizers.Nadam(learning_rate=lr),
#              metrics=[keras.metrics.sparse_categorical_accuracy]) # !

def custom_training_loop():
    def shuffle(X, y):
        assert len(X) == len(y)
        idx = np.random.permutation(len(X))
        return X[idx], y[idx]
    
    n_epochs = 5
    batch_size = 32
    optimizer=keras.optimizers.Nadam(learning_rate=0.001)
    loss_fn = keras.losses.sparse_categorical_crossentropy # try class
    metrics=[keras.metrics.SparseCategoricalAccuracy] # try func
    model = build_simple_model()
    
    for epoch in range(1, n_epochs + 1):
        log.info(f'Epoch {epoch}...')
        n_steps = len(X) // 32 + 1
        X_shuffled, y_shuffled = shuffle(X, y_sparse)
        for step in range(1, n_steps + 1):
            ceiling = step * batch_size if step * batch_size < len(X_shuffled) else len(X_shuffled)
            floor = (step - 1) * batch_size
            X_batch, y_batch = X_shuffled[floor: ceiling], y_shuffled[floor: ceiling]
            with tf.GradientTape() as tape:
                y_predicted = model(X_batch)
                print(y_predicted)
            break
        break
            # compute grad
            # autodiff
            # loss, metric
            
custom_training_loop()

2021-03-29 20:22:33,735 : INFO : Epoch 1...


tf.Tensor(
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.75778348e-07 0.00000000e+00 0.00000000e+00 0.00000000e+00
  9.99999762e-01 0.00000000e+00]
 [0.00000000e+00 1.30162815e-20 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.05102732e-12 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.09359920e-02 0.00000000e+00 0.00000000e+00
  9.89064038e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  9.72668505e-38 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.02959250e-08 3.98707125e-37
  1.30991805e-18 0.00000000e+00 0.00000000e+00 3.05706194e-23
  1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 7.73479140e-34 0.00000000e+00
  2.64196661e-22 0.00000000e+00 0.00000000e+00 1.96843581e-21
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 